In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
from pathlib import Path
import re 

import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid

root_path = Path('/dev/projects/cds/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)

from api.blpw import BlpQuery
from api.data.base import data_path
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.pricing.utils import (
    get_cds_specs,
    get_cds_curve,
    get_cds_spreads,
    get_risk_free_curve,
    get_cds_implied_price,
    get_curve,
)

In [3]:
# Technically, the model requires the discount factor to be 1 at spot;
# but we can't do that and also have the discount curve extend back to
# the trade date.  For the time being, we'll keep discount = 1 at trade.
# The results match anyway.

#### Local data

In [4]:
curve_hist = pd.read_csv(data_path / 'curve_history.csv', parse_dates=['date'], dayfirst=True)
cds_ref = pd.read_csv(data_path / 'cds_reference.csv', parse_dates=['maturity'], dayfirst=True)
cds_hist = pd.read_csv(data_path / 'cds_history.csv', parse_dates=['date'], dayfirst=True)

In [6]:
cds_hist

date         security  last_price  \
0      2007-01-01  CPETR1U5 Curncy        13.0   
1      2007-01-01   CPAN1U3 Curncy        38.5   
2      2007-01-01   CMEX1U1 Curncy        13.0   
3      2007-01-01  CVENZ1U3 Curncy        74.0   
4      2007-01-01   CPAN1U5 Curncy        84.0   
...           ...              ...         ...   
666006 2022-06-23  CT971833 Curncy       654.5   
666007 2022-06-23  CPHIL1U5 Curncy       122.0   
666008 2022-06-23  CSOAF1U3 Curncy       221.0   
666009 2022-06-23  CBULG1U5 Curncy        93.5   
666010 2022-06-23   CEGY1U1 Curncy       834.5   

                                   name  
0        PETMK CDS USD SR 5Y D14 Curncy  
1       PANAMA CDS USD SR 3Y D14 Curncy  
2          MEX CDS USD SR 1Y D14 Curncy  
3         VENZ CDS USD SR 3Y D14 Curncy  
4       PANAMA CDS USD SR 5Y D14 Curncy  
...                                 ...  
666006   ESKOM CDS USD SR 7Y D14 Curncy  
666007  PHILIP CDS USD SR 5Y D14 Curncy  
666008  REPSOU CDS USD SR 3Y D14 Curncy  
666009  BGARIA CDS USD SR 5Y D14 Curncy  
666010   EGYPT CDS USD SR 1Y D14 Curncy  

[666011 rows x 4 columns]

#### Settings

In [8]:
cds_name = 'TURKEY CDS USD SR 5Y D14 Curncy'
index_name = 'CDX EM CDSI S37 5Y PRC Corp'
curve_name = 'USD ISDA CDS SOFR Rates Swap Curve' #  "USD ISDA Standard Rates Swap Curve" # "EUR ISDA Standard Rates Swap Curve"
trade_date = pd.Timestamp.today().floor('d')
pcs = 'MSG1'
sep = ' '

date_start = pd.Timestamp('2021-06-02')
date_end = pd.Timestamp('2022-06-23')

dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

#### Bloomberg Connection

In [9]:
bq = BlpQuery(timeout=50000).start()

#### Index Members

In [ ]:
members = bq.bds(
    security=index_name,
    field='indx_members'
).iloc[:, :-1].set_axis(['issuer', 'wgt', 'currency', 'seniority', 'security'], axis=1)
members['security'] = members['security'] + ' Curncy'

#### Current Spreads

In [10]:
fields = [
    'security_des',
    'issuer_bulk',
    'crncy',
    'cds_recovery_rt',
    'tenor',
    'upfront_format_indicator',
    'px_last',
    'upfront_last'
    'sw_spread',
    # '5y_mid_cds_spread',
]

In [ ]:
securities = members['security']
securities = securities.to_list() if not pcs else add_pcs(securities, source=pcs, sep=sep)
cds_spreads = bq.bdp(
    securities=securities,
    fields=fields
).rename(
    {
        'crncy': 'currency', 
        'cds_recovery_rt': 'recovery',
        'sw_spread': 'coupon',
        'issuer_bulk': 'issuer'
    }, axis=1)

In [ ]:
def parse_tenor(tenor, key='tenor'):
    fn = lambda tenor: f"{tenor:.0f}Y" if tenor > 1 else f"{int(12 * tenor):.0f}M"
    if isinstance(tenor, pd.DataFrame):
        return tenor.assign(tenor=tenor[key].map(fn))
    elif isinstance(tenor, pd.Series):
        return tenor.map(fn)
    elif isinstance(tenor, str):
        return fn(tenor)
    else:
        raise TypeError(f'Only str, DataFrame od Series are allowed, got object of type {type(tenor)} instead')

In [ ]:
parse_tenor(cds_spreads['tenor'])

In [ ]:
cds_spreads['tenor'].map(lambda tenor: f"{tenor:.0f}Y" if tenor > 1 else f"{int(12 * tenor):.0f}M")

In [ ]:
tenor = 

In [ ]:
cds_spreads

In [ ]:
if pcs:
    cds_spreads['security'] = cds_spreads['security'].str.replace(f"{sep}{pcs}", '')
cds_spreads = cds_spreads.merge(members[['security', 'wgt']], on='security', how='left')

#### Current Risk Free Curve

In [ ]:
curve = get_curve(curve_name, curves)
curve_rates = bq.bdp(
    securities=[memb['ticker'] for memb in curve['members']],
    fields=['px_last']
)# .assign(tenor=[memb['tenor'] for memb in curve['members']])
curve_rates['tenor'] = curve_rates['security'].replace({memb['ticker']: memb['tenor'] for memb in curve['members']})

In [ ]:
# cds_specs = get_cds_specs(cds_name, cds_ref)
# cds_curve = get_cds_curve(cds_name, cds_ref, cds_hist)
# cds_spread  = get_cds_spreads(cds_name, cds_hist).loc[date_start:date_end]
risk_free_curve = curve_rates.set_index('tenor')['px_last']
notional = 10000000
coupon=100
flat = True
n =10

#### Pricing

In [ ]:
data = pd.read_clipboard()
data = cds_spreads

In [ ]:
res = []
for index, cds_specs in data.iterrows():
    spread = cds_specs['px_last']
    coupon = cds_specs['coupon']
    recovery = cds_specs['recovery']
    # upfront = cds_specs['upfront']
    # notional = cds_specs['wgt']
    cds = get_isda_cds(
        risk_free_curve=risk_free_curve,
        trade_date=trade_date,
        cds_spread=spread,
        recovery=recovery,
        upfront=0,
        notional=notional,
        coupon=coupon,
        rebates_accrual=True
    )
    
    res.append({'npv': cds.NPV(), 'cds_implied_price': 100 * (1+cds.fairUpfront()), 'spread': spread})

### Results

In [ ]:
res = pd.DataFrame(res).assign(
    issuer=data['issuer'],
    security=data['security'],
    wgt=data['wgt'],
    
)

In [ ]:
f"intrinsic value: {((res['wgt'] * res['cds_implied_price']) / res['wgt'].sum()).sum()}"